In [144]:
import pandas as pd
import numpy as np
import hvplot.pandas
from sklearn.cluster import KMeans
import plotly.express as px
import psycopg2 as pg
import sqlalchemy
import matplotlib as plt
from sqlalchemy import create_engine
import pandas.io.sql as psql
import sklearn as skl
import tensorflow as tf

In [145]:
#Read Incident dataframe from postgreSQL (data was cleaned in this file, saved and imported into PostgreSQL, and then read in here as DataFrame)

connection = pg.connect(host='team-11.ch3wmigorcf0.us-east-2.rds.amazonaws.com', dbname="postgres", user="postgres", password='UTB00tcamp')
#dataframe = psql.DataFrame("SELECT * FROM category", connection)
VAERS_incidents_postgres = pd.read_sql_query('select * from public."INCIDENT"',con=connection)
VAERS_incidents_postgres.head()

,PRIMARY_KEY,VAERS_ID,AGE_YRS,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,GENDER_0,GENDER_1
0,1,855017,55,0,0,1,1,1,1,1,0
1,2,855018,68,0,0,1,1,1,1,1,0
2,3,855019,50,0,0,0,0,0,0,1,0
3,4,855020,67,0,0,1,1,1,1,1,0
4,5,855021,73,0,0,0,0,0,0,1,0


In [146]:
#Set Primary Key as Index
VAERS_incidents_postgres.set_index("PRIMARY_KEY", inplace= True)
VAERS_incidents_postgres.head()

,VAERS_ID,AGE_YRS,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,GENDER_0,GENDER_1
PRIMARY_KEY,,,,,,,,,,
1,855017,55,0,0,1,1,1,1,1,0
2,855018,68,0,0,1,1,1,1,1,0
3,855019,50,0,0,0,0,0,0,1,0
4,855020,67,0,0,1,1,1,1,1,0
5,855021,73,0,0,0,0,0,0,1,0


In [147]:
## Reading in original data to begin cleaning
file_path_data = "Resources/Data/2021VAERSDATA.csv"
encoding = 'unicode_escape'
VAERS_DATA_df = pd.read_csv(file_path_data, encoding='unicode_escape')
VAERS_DATA_df=VAERS_DATA_df.set_index(["VAERS_ID"]).copy()
VAERS_DATA_df.head()

,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,DIED,DATEDIED,L_THREAT,ER_VISIT,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
VAERS_ID,,,,,,,,,,,,,,,,,,,,,
916710,1/1/2021,MO,23.0,23.0,NaN,F,NaN,NaN,Y,NaN,...,NaN,Hypothyroidism,NaN,NaN,2,1/1/2021,NaN,NaN,Y,NKDA
916720,1/1/2021,NY,23.0,23.0,NaN,M,NaN,NaN,NaN,NaN,...,NaN,NaN,unknown,NaN,2,1/1/2021,NaN,NaN,Y,NaN
916741,1/1/2021,AR,68.0,68.0,NaN,F,NaN,NaN,NaN,NaN,...,had surgery R hand for advanced arthritis 11/1...,Rheumatoid arthritis - mostly affecting R wris...,NaN,NaN,2,1/1/2021,NaN,NaN,NaN,bee stings
916772,1/1/2021,GA,55.0,55.0,NaN,M,NaN,NaN,NaN,NaN,...,NaN,"HTN, Insomnia,High Cholesterol,",NaN,NaN,2,1/1/2021,NaN,Y,NaN,Codeine
916790,1/1/2021,TN,52.0,52.0,NaN,F,NaN,NaN,Y,NaN,...,NaN,"Asthma, migraines",NaN,NaN,2,1/1/2021,NaN,NaN,Y,"Sulfa, shellfish"


In [148]:
file_path_vax = "Resources/Data/2021VAERSVAX.csv"
encoding = 'unicode_escape'
VAERS_VAX_df = pd.read_csv(file_path_vax, encoding= 'unicode_escape')
VAERS_VAX_df=VAERS_VAX_df.set_index(["VAERS_ID"]).copy()
VAERS_VAX_df.head()

,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
VAERS_ID,,,,,,,
916710,COVID19,MODERNA,NaN,1,IM,LA,COVID19 (COVID19 (MODERNA))
916720,COVID19,MODERNA,011L20A,1,IM,AR,COVID19 (COVID19 (MODERNA))
916741,COVID19,PFIZER\BIONTECH,EH9899,1,SYR,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
916772,COVID19,PFIZER\BIONTECH,EJ1685,UNK,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
916790,COVID19,PFIZER\BIONTECH,NaN,1,IM,RA,COVID19 (COVID19 (PFIZER-BIONTECH))


In [149]:
VAERS_VAX_df=VAERS_VAX_df[VAERS_VAX_df.VAX_TYPE == "COVID19"].copy()
VAERS_VAX_df.head()

,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
VAERS_ID,,,,,,,
916710,COVID19,MODERNA,NaN,1,IM,LA,COVID19 (COVID19 (MODERNA))
916720,COVID19,MODERNA,011L20A,1,IM,AR,COVID19 (COVID19 (MODERNA))
916741,COVID19,PFIZER\BIONTECH,EH9899,1,SYR,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
916772,COVID19,PFIZER\BIONTECH,EJ1685,UNK,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
916790,COVID19,PFIZER\BIONTECH,NaN,1,IM,RA,COVID19 (COVID19 (PFIZER-BIONTECH))


In [150]:
#Drop Unnecessary columns from VAERS_VAX_df
VAERS_VAX_df= VAERS_VAX_df.drop(["VAX_TYPE", "VAX_LOT", "VAX_ROUTE", "VAX_SITE", "VAX_NAME"], axis=1).copy()
VAERS_VAX_df.head()

,VAX_MANU,VAX_DOSE_SERIES
VAERS_ID,,
916710,MODERNA,1
916720,MODERNA,1
916741,PFIZER\BIONTECH,1
916772,PFIZER\BIONTECH,UNK
916790,PFIZER\BIONTECH,1


In [151]:
file_path_symptoms = "Resources/Data/2021VAERSSYMPTOMS.csv"
encoding = 'unicode_escape'
VAERS_SYMPTOMS_df = pd.read_csv(file_path_symptoms, encoding= 'unicode_escape')
VAERS_SYMPTOMS_df.head()

,VAERS_ID,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0,916710,Appendicitis,23.1,Band neutrophil percentage increased,23.1,Surgery,23.1,White blood cell count increased,23.1,NaN,NaN
1,916720,Dysphagia,23.1,Dyspnoea,23.1,NaN,NaN,NaN,NaN,NaN,NaN
2,916741,Chills,23.1,Complex regional pain syndrome,23.1,Fatigue,23.1,Headache,23.1,Joint range of motion decreased,23.1
3,916741,Myalgia,23.1,Pain in extremity,23.1,Peripheral swelling,23.1,X-ray abnormal,23.1,NaN,NaN
4,916772,Feeling abnormal,23.1,SARS-CoV-2 test positive,23.1,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
VAERS_DATA_df.columns

Index(['RECVDATE', 'STATE', 'AGE_YRS', 'CAGE_YR', 'CAGE_MO', 'SEX', 'DIED',
       'DATEDIED', 'L_THREAT', 'ER_VISIT', 'HOSPITAL', 'HOSPDAYS', 'X_STAY',
       'DISABLE', 'RECOVD', 'VAX_DATE', 'ONSET_DATE', 'NUMDAYS', 'LAB_DATA',
       'V_ADMINBY', 'V_FUNDBY', 'OTHER_MEDS', 'CUR_ILL', 'HISTORY',
       'PRIOR_VAX', 'SPLTTYPE', 'FORM_VERS', 'TODAYS_DATE', 'BIRTH_DEFECT',
       'OFC_VISIT', 'ER_ED_VISIT', 'ALLERGIES'],
      dtype='object')

In [153]:
VAERS_DATA_df=VAERS_DATA_df.drop(['RECVDATE', 'STATE', 'CAGE_YR', 'CAGE_MO', 'LAB_DATA', 'DATEDIED', 'L_THREAT', 'ER_VISIT', 'HOSPDAYS', 'X_STAY', 'VAX_DATE', 'V_ADMINBY', 'V_FUNDBY', 'ER_ED_VISIT', 'OFC_VISIT', 'FORM_VERS', 'BIRTH_DEFECT', 'DISABLE', 'RECOVD', 'PRIOR_VAX', 'ONSET_DATE', 'NUMDAYS', 'SPLTTYPE', 'TODAYS_DATE'], axis=1).copy()
VAERS_DATA_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
916710,23.0,F,NaN,Y,Synthroid,NaN,Hypothyroidism,NKDA
916720,23.0,M,NaN,NaN,NaN,NaN,NaN,NaN
916741,68.0,F,NaN,NaN,phenobarbital 60mg HS hydroxychloroquin 400mg...,had surgery R hand for advanced arthritis 11/1...,Rheumatoid arthritis - mostly affecting R wris...,bee stings
916772,55.0,M,NaN,NaN,"Propranolol,zoloft,asa,Lisinipril, Crestor, Pr...",NaN,"HTN, Insomnia,High Cholesterol,",Codeine
916790,52.0,F,NaN,NaN,"Estradiol, topiramate, Emgality, sumatriptan, ...",NaN,"Asthma, migraines","Sulfa, shellfish"


In [154]:
VAERS_DATA_df.columns

Index(['AGE_YRS', 'SEX', 'DIED', 'HOSPITAL', 'OTHER_MEDS', 'CUR_ILL',
       'HISTORY', 'ALLERGIES'],
      dtype='object')

In [155]:
VAERS_DATA_df.dtypes

AGE_YRS       float64
SEX            object
DIED           object
HOSPITAL       object
OTHER_MEDS     object
CUR_ILL        object
HISTORY        object
ALLERGIES      object
dtype: object

In [156]:
#Transform String Columns
def change_string(died):
    if died == "Y":
        return 1
    else:
        return 0 

VAERS_DATA_df["DIED"]= VAERS_DATA_df["DIED"].apply(change_string)
VAERS_DATA_df.head(20)

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
916710,23.00,F,0,Y,Synthroid,NaN,Hypothyroidism,NKDA
916720,23.00,M,0,NaN,NaN,NaN,NaN,NaN
916741,68.00,F,0,NaN,phenobarbital 60mg HS hydroxychloroquin 400mg...,had surgery R hand for advanced arthritis 11/1...,Rheumatoid arthritis - mostly affecting R wris...,bee stings
916772,55.00,M,0,NaN,"Propranolol,zoloft,asa,Lisinipril, Crestor, Pr...",NaN,"HTN, Insomnia,High Cholesterol,",Codeine
916790,52.00,F,0,NaN,"Estradiol, topiramate, Emgality, sumatriptan, ...",NaN,"Asthma, migraines","Sulfa, shellfish"
916803,78.00,M,1,Y,"Lisinopril, Novolog,, Lantus Solostar, Gabapen...",none,"large T-cell lymphoma, HTN, Gout, recieving t...",N?A
916809,40.00,F,0,Y,NaN,None,NaN,None
916836,55.00,M,0,Y,No current medications,Pt reported frequent urination for several months,None,No known allergies
916859,37.00,F,0,NaN,"fexofenadine, famotidine, hydroxychloroquine,",None,"Undifferentiated connective tissue disorder, a...","peanuts, soy, oysters, crab/lobster, penicillin"


In [157]:
#Transform String Columns
def change_string1(hospital):
    if hospital == "Y":
        return 1
    else:
        return 0 

VAERS_DATA_df["HOSPITAL"]= VAERS_DATA_df["HOSPITAL"].apply(change_string1)
VAERS_DATA_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
916710,23.0,F,0,1,Synthroid,NaN,Hypothyroidism,NKDA
916720,23.0,M,0,0,NaN,NaN,NaN,NaN
916741,68.0,F,0,0,phenobarbital 60mg HS hydroxychloroquin 400mg...,had surgery R hand for advanced arthritis 11/1...,Rheumatoid arthritis - mostly affecting R wris...,bee stings
916772,55.0,M,0,0,"Propranolol,zoloft,asa,Lisinipril, Crestor, Pr...",NaN,"HTN, Insomnia,High Cholesterol,",Codeine
916790,52.0,F,0,0,"Estradiol, topiramate, Emgality, sumatriptan, ...",NaN,"Asthma, migraines","Sulfa, shellfish"


In [158]:
#Transform String Columns
VAERS_DATA_df['OTHER_MEDS'] = VAERS_DATA_df['OTHER_MEDS'].notnull().copy()
VAERS_DATA_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
916710,23.0,F,0,1,True,NaN,Hypothyroidism,NKDA
916720,23.0,M,0,0,False,NaN,NaN,NaN
916741,68.0,F,0,0,True,had surgery R hand for advanced arthritis 11/1...,Rheumatoid arthritis - mostly affecting R wris...,bee stings
916772,55.0,M,0,0,True,NaN,"HTN, Insomnia,High Cholesterol,",Codeine
916790,52.0,F,0,0,True,NaN,"Asthma, migraines","Sulfa, shellfish"


In [159]:
VAERS_DATA_df['OTHER_MEDS']=VAERS_DATA_df['OTHER_MEDS'].map({False: 0, True: 1}).copy()
VAERS_DATA_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
916710,23.0,F,0,1,1,NaN,Hypothyroidism,NKDA
916720,23.0,M,0,0,0,NaN,NaN,NaN
916741,68.0,F,0,0,1,had surgery R hand for advanced arthritis 11/1...,Rheumatoid arthritis - mostly affecting R wris...,bee stings
916772,55.0,M,0,0,1,NaN,"HTN, Insomnia,High Cholesterol,",Codeine
916790,52.0,F,0,0,1,NaN,"Asthma, migraines","Sulfa, shellfish"


In [160]:
#Transform String Columns
VAERS_DATA_df['CUR_ILL'] = VAERS_DATA_df['CUR_ILL'].notnull().copy()
VAERS_DATA_df['CUR_ILL']=VAERS_DATA_df['CUR_ILL'].map({False: 0, True: 1}).copy()
VAERS_DATA_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
916710,23.0,F,0,1,1,0,Hypothyroidism,NKDA
916720,23.0,M,0,0,0,0,NaN,NaN
916741,68.0,F,0,0,1,1,Rheumatoid arthritis - mostly affecting R wris...,bee stings
916772,55.0,M,0,0,1,0,"HTN, Insomnia,High Cholesterol,",Codeine
916790,52.0,F,0,0,1,0,"Asthma, migraines","Sulfa, shellfish"


In [161]:
#Transform String Columns
VAERS_DATA_df['HISTORY'] = VAERS_DATA_df['HISTORY'].notnull().copy()
VAERS_DATA_df['HISTORY']=VAERS_DATA_df['HISTORY'].map({False: 0, True: 1}).copy()
VAERS_DATA_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
916710,23.0,F,0,1,1,0,1,NKDA
916720,23.0,M,0,0,0,0,0,NaN
916741,68.0,F,0,0,1,1,1,bee stings
916772,55.0,M,0,0,1,0,1,Codeine
916790,52.0,F,0,0,1,0,1,"Sulfa, shellfish"


In [162]:
#Transform String Columns
VAERS_DATA_df['ALLERGIES'] = VAERS_DATA_df['ALLERGIES'].notnull().copy()
VAERS_DATA_df['ALLERGIES']=VAERS_DATA_df['ALLERGIES'].map({False: 0, True: 1}).copy()
VAERS_DATA_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
916710,23.0,F,0,1,1,0,1,1
916720,23.0,M,0,0,0,0,0,0
916741,68.0,F,0,0,1,1,1,1
916772,55.0,M,0,0,1,0,1,1
916790,52.0,F,0,0,1,0,1,1


In [163]:
##Read in 2020 data to clean and merge with 2021 data

In [164]:
file_path_2020data = "Resources/Data/2020VAERSDATA.csv"
encoding = 'unicode_escape'
VAERS_DATA_20_df = pd.read_csv(file_path_2020data, encoding='unicode_escape')
VAERS_DATA_20_df = VAERS_DATA_20_df.set_index(["VAERS_ID"]).copy()
VAERS_DATA_20_df.head()

,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,DATEDIED,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
VAERS_ID,,,,,,,,,,,,,,,,,,,,,
855017,01/01/2020,HI,55.0,55.0,NaN,F,NaN,Symptoms occurred almost immediately- aching j...,NaN,NaN,...,None,Migraines,NaN,NaN,2,01/01/2020,NaN,NaN,NaN,Morphine
855018,01/01/2020,WI,68.0,68.0,NaN,F,NaN,"Extreme pain, muscle weakness in right arm so ...",NaN,NaN,...,None,"Hypothyroidism, Meniere's Disease.",NaN,NaN,2,01/01/2020,NaN,Y,Y,"Penicillin, sulfa drugs, erythromycin, tetracy..."
855019,01/01/2020,NaN,50.0,50.0,NaN,F,NaN,SORENESS IN THE AREA. ITCHING AND RASH,NaN,NaN,...,NaN,NaN,NaN,NaN,2,01/01/2020,NaN,NaN,NaN,NaN
855020,01/01/2020,TX,67.0,67.0,NaN,F,NaN,"sore arm, fever 101 , chills, and aching musc...",NaN,NaN,...,none,none,NaN,NaN,2,01/01/2020,NaN,NaN,NaN,"morphine,"
855021,01/01/2020,NaN,73.0,73.0,NaN,F,NaN,Patient presented to Clinic today 1/1/2020. Sh...,NaN,NaN,...,NaN,NaN,NaN,NaN,2,01/01/2020,NaN,Y,NaN,NaN


In [165]:
file_path_2020symptoms = "Resources/Data/2020VAERSSYMPTOMS.csv"
encoding = 'unicode_escape'
VAERS_SYMPTOMS_20_df = pd.read_csv(file_path_2020symptoms, encoding='unicode_escape')
VAERS_SYMPTOMS_20_df.head()

,VAERS_ID,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0,855017,Arthralgia,22.1,Chills,22.1,Injection site pain,22.1,Pyrexia,22.1,NaN,NaN
1,855018,Chills,22.1,Fatigue,22.1,Hypertension,22.1,Hypoaesthesia,22.1,Injected limb mobility decreased,22.1
2,855018,Muscular weakness,22.1,Pain in extremity,22.1,Pyrexia,22.1,Tremor,22.1,Vertigo,22.1
3,855019,Pain,22.1,Pruritus,22.1,Rash,22.1,NaN,NaN,NaN,NaN
4,855020,Chills,22.1,Influenza like illness,22.1,Myalgia,22.1,Pain in extremity,22.1,Pyrexia,22.1


In [166]:
file_path_2020vax = "Resources/Data/2020VAERSvax.csv"
encoding = 'unicode_escape'
VAERS_VAX_20_df = pd.read_csv(file_path_2020vax, encoding='unicode_escape')
VAERS_VAX_20_df= VAERS_VAX_20_df.set_index(["VAERS_ID"]).copy()
VAERS_VAX_20_df.head()

,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
VAERS_ID,,,,,,,
855017,VARZOS,GLAXOSMITHKLINE BIOLOGICALS,NaN,2,SYR,LA,ZOSTER (SHINGRIX)
855018,UNK,UNKNOWN MANUFACTURER,NaN,1,IM,RA,VACCINE NOT SPECIFIED (OTHER)
855019,VARZOS,GLAXOSMITHKLINE BIOLOGICALS,NaN,UNK,NaN,NaN,ZOSTER (SHINGRIX)
855020,VARZOS,GLAXOSMITHKLINE BIOLOGICALS,NaN,UNK,IM,LA,ZOSTER (SHINGRIX)
855021,VARZOS,GLAXOSMITHKLINE BIOLOGICALS,NaN,2,IM,RA,ZOSTER (SHINGRIX)


In [167]:
#Begin to clean 2020 dataframes to merge later with 2021
VAERS_DATA_20_df=VAERS_DATA_20_df.drop(['RECVDATE', 'RPT_DATE', 'SYMPTOM_TEXT', 'STATE', 'CAGE_YR', 'CAGE_MO', 'LAB_DATA', 'DATEDIED', 'L_THREAT', 'ER_VISIT', 'HOSPDAYS', 'X_STAY', 'VAX_DATE', 'V_ADMINBY', 'V_FUNDBY', 'ER_ED_VISIT', 'OFC_VISIT', 'FORM_VERS', 'BIRTH_DEFECT', 'DISABLE', 'RECOVD', 'PRIOR_VAX', 'ONSET_DATE', 'NUMDAYS', 'SPLTTYPE', 'TODAYS_DATE'], axis=1).copy()
VAERS_DATA_20_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
855017,55.0,F,NaN,NaN,Loristan,None,Migraines,Morphine
855018,68.0,F,NaN,NaN,"levothyroxine, multi vitamin, calcium.",None,"Hypothyroidism, Meniere's Disease.","Penicillin, sulfa drugs, erythromycin, tetracy..."
855019,50.0,F,NaN,NaN,NaN,NaN,NaN,NaN
855020,67.0,F,NaN,NaN,"multi vitamin, calcium, premarin cream",none,none,"morphine,"
855021,73.0,F,NaN,NaN,NaN,NaN,NaN,NaN


In [168]:
#Transform String Columns
def transform_string(died):
    if died == "Y":
        return 1
    else:
        return 0 

VAERS_DATA_20_df["DIED"]= VAERS_DATA_20_df["DIED"].apply(transform_string)
VAERS_DATA_20_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
855017,55.0,F,0,NaN,Loristan,None,Migraines,Morphine
855018,68.0,F,0,NaN,"levothyroxine, multi vitamin, calcium.",None,"Hypothyroidism, Meniere's Disease.","Penicillin, sulfa drugs, erythromycin, tetracy..."
855019,50.0,F,0,NaN,NaN,NaN,NaN,NaN
855020,67.0,F,0,NaN,"multi vitamin, calcium, premarin cream",none,none,"morphine,"
855021,73.0,F,0,NaN,NaN,NaN,NaN,NaN


In [169]:
#Transform String Columns
def transform_string1(hospital):
    if hospital == "Y":
        return 1
    else:
        return 0 

VAERS_DATA_20_df["HOSPITAL"]= VAERS_DATA_20_df["HOSPITAL"].apply(transform_string1)


In [170]:
#Transform String Columns

VAERS_DATA_20_df['OTHER_MEDS'] = VAERS_DATA_20_df['OTHER_MEDS'].notnull().copy()
VAERS_DATA_20_df['OTHER_MEDS']=VAERS_DATA_20_df['OTHER_MEDS'].map({False: 0, True: 1}).copy()

VAERS_DATA_20_df['CUR_ILL'] = VAERS_DATA_20_df['CUR_ILL'].notnull().copy()
VAERS_DATA_20_df['CUR_ILL']=VAERS_DATA_20_df['CUR_ILL'].map({False: 0, True: 1}).copy()

VAERS_DATA_20_df['HISTORY'] = VAERS_DATA_20_df['HISTORY'].notnull().copy()
VAERS_DATA_20_df['HISTORY']=VAERS_DATA_20_df['HISTORY'].map({False: 0, True: 1}).copy()

VAERS_DATA_20_df['ALLERGIES'] = VAERS_DATA_20_df['ALLERGIES'].notnull().copy()
VAERS_DATA_20_df['ALLERGIES']=VAERS_DATA_20_df['ALLERGIES'].map({False: 0, True: 1}).copy()

VAERS_DATA_20_df.head()


,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
855017,55.0,F,0,0,1,1,1,1
855018,68.0,F,0,0,1,1,1,1
855019,50.0,F,0,0,0,0,0,0
855020,67.0,F,0,0,1,1,1,1
855021,73.0,F,0,0,0,0,0,0


In [171]:
#Drop Unnecessary columns from VAERS_VAX_20_df
VAERS_VAX_20_df= VAERS_VAX_20_df.drop(["VAX_TYPE", "VAX_LOT", "VAX_ROUTE", "VAX_SITE", "VAX_NAME"], axis=1).copy()

VAERS_VAX_20_df.head()

,VAX_MANU,VAX_DOSE_SERIES
VAERS_ID,,
855017,GLAXOSMITHKLINE BIOLOGICALS,2
855018,UNKNOWN MANUFACTURER,1
855019,GLAXOSMITHKLINE BIOLOGICALS,UNK
855020,GLAXOSMITHKLINE BIOLOGICALS,UNK
855021,GLAXOSMITHKLINE BIOLOGICALS,2


In [172]:
len(VAERS_VAX_df.index)

5279

In [173]:
len(VAERS_DATA_df.index)

5351

In [174]:
VAERS_DATA_VAX_df= pd.merge(VAERS_DATA_df, VAERS_VAX_df, on = "VAERS_ID")
VAERS_DATA_VAX_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,VAX_MANU,VAX_DOSE_SERIES
VAERS_ID,,,,,,,,,,
916710,23.0,F,0,1,1,0,1,1,MODERNA,1
916720,23.0,M,0,0,0,0,0,0,MODERNA,1
916741,68.0,F,0,0,1,1,1,1,PFIZER\BIONTECH,1
916772,55.0,M,0,0,1,0,1,1,PFIZER\BIONTECH,UNK
916790,52.0,F,0,0,1,0,1,1,PFIZER\BIONTECH,1


In [175]:
VAERS_DATA_VAX_20_df = pd.merge(VAERS_DATA_20_df, VAERS_VAX_20_df, on="VAERS_ID")
VAERS_DATA_VAX_20_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,VAX_MANU,VAX_DOSE_SERIES
VAERS_ID,,,,,,,,,,
855017,55.0,F,0,0,1,1,1,1,GLAXOSMITHKLINE BIOLOGICALS,2
855018,68.0,F,0,0,1,1,1,1,UNKNOWN MANUFACTURER,1
855019,50.0,F,0,0,0,0,0,0,GLAXOSMITHKLINE BIOLOGICALS,UNK
855020,67.0,F,0,0,1,1,1,1,GLAXOSMITHKLINE BIOLOGICALS,UNK
855021,73.0,F,0,0,0,0,0,0,GLAXOSMITHKLINE BIOLOGICALS,2


In [176]:
##2020 AND 2021 VAERS DATA and VAX dataframes combined
VAERS_DATA_VAX_all_df = VAERS_DATA_VAX_20_df.append(VAERS_DATA_VAX_df)
VAERS_DATA_VAX_all_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,VAX_MANU,VAX_DOSE_SERIES
VAERS_ID,,,,,,,,,,
855017,55.0,F,0,0,1,1,1,1,GLAXOSMITHKLINE BIOLOGICALS,2
855018,68.0,F,0,0,1,1,1,1,UNKNOWN MANUFACTURER,1
855019,50.0,F,0,0,0,0,0,0,GLAXOSMITHKLINE BIOLOGICALS,UNK
855020,67.0,F,0,0,1,1,1,1,GLAXOSMITHKLINE BIOLOGICALS,UNK
855021,73.0,F,0,0,0,0,0,0,GLAXOSMITHKLINE BIOLOGICALS,2


In [177]:
# Drop VAX_MANU Column
VAERS_DATA_VAX_all_df = VAERS_DATA_VAX_all_df.drop(["VAX_MANU"], axis=1).copy()
VAERS_DATA_VAX_all_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,VAX_DOSE_SERIES
VAERS_ID,,,,,,,,,
855017,55.0,F,0,0,1,1,1,1,2
855018,68.0,F,0,0,1,1,1,1,1
855019,50.0,F,0,0,0,0,0,0,UNK
855020,67.0,F,0,0,1,1,1,1,UNK
855021,73.0,F,0,0,0,0,0,0,2


In [178]:
VAERS_DATA_VAX_all_df.dtypes

AGE_YRS            float64
SEX                 object
DIED                 int64
HOSPITAL             int64
OTHER_MEDS           int64
CUR_ILL              int64
HISTORY              int64
ALLERGIES            int64
VAX_DOSE_SERIES     object
dtype: object

In [179]:
# Dropping VAX_DOSE_SERIES for now, will add back later
VAERS_DATA_VAX_all_df = VAERS_DATA_VAX_all_df.drop(["VAX_DOSE_SERIES"], axis=1).copy()
VAERS_DATA_VAX_all_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
855017,55.0,F,0,0,1,1,1,1
855018,68.0,F,0,0,1,1,1,1
855019,50.0,F,0,0,0,0,0,0
855020,67.0,F,0,0,1,1,1,1
855021,73.0,F,0,0,0,0,0,0


In [180]:
#Transform String Column "SEX"
def no_string(gender):
    if gender == "M":
        return 1
    else:
        return 0 

VAERS_DATA_VAX_all_df["SEX"]= VAERS_DATA_VAX_all_df["SEX"].apply(no_string)
VAERS_DATA_VAX_all_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
855017,55.0,0,0,0,1,1,1,1
855018,68.0,0,0,0,1,1,1,1
855019,50.0,0,0,0,0,0,0,0
855020,67.0,0,0,0,1,1,1,1
855021,73.0,0,0,0,0,0,0,0


In [181]:
VAERS_DATA_VAX_all_df = pd.get_dummies(VAERS_DATA_VAX_all_df, prefix=["GENDER"], columns= ["SEX"]).copy()
VAERS_DATA_VAX_all_df.head()

,AGE_YRS,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,GENDER_0,GENDER_1
VAERS_ID,,,,,,,,,
855017,55.0,0,0,1,1,1,1,1,0
855018,68.0,0,0,1,1,1,1,1,0
855019,50.0,0,0,0,0,0,0,1,0
855020,67.0,0,0,1,1,1,1,1,0
855021,73.0,0,0,0,0,0,0,1,0


In [182]:
VAERS_DATA_VAX_all_df.AGE_YRS=VAERS_DATA_VAX_all_df.AGE_YRS.fillna(0)

In [183]:
VAERS_DATA_VAX_all_df.AGE_YRS=VAERS_DATA_VAX_all_df.AGE_YRS.astype(int)

In [184]:
VAERS_DATA_VAX_all_df.dtypes

AGE_YRS       int32
DIED          int64
HOSPITAL      int64
OTHER_MEDS    int64
CUR_ILL       int64
HISTORY       int64
ALLERGIES     int64
GENDER_0      uint8
GENDER_1      uint8
dtype: object

In [185]:
np.isfinite(VAERS_DATA_VAX_all_df).all()

AGE_YRS       True
DIED          True
HOSPITAL      True
OTHER_MEDS    True
CUR_ILL       True
HISTORY       True
ALLERGIES     True
GENDER_0      True
GENDER_1      True
dtype: bool

In [186]:
### Logistic Regression Model to predict death
y = VAERS_DATA_VAX_all_df["DIED"]
X = VAERS_DATA_VAX_all_df.drop(["DIED"], axis=1)
X.head()

,AGE_YRS,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,GENDER_0,GENDER_1
VAERS_ID,,,,,,,,
855017,55,0,1,1,1,1,1,0
855018,68,0,1,1,1,1,1,0
855019,50,0,0,0,0,0,1,0
855020,67,0,1,1,1,1,1,0
855021,73,0,0,0,0,0,1,0


In [187]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y, random_state=1, stratify=y)

In [188]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
   max_iter=200,
   random_state=1)

In [189]:
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [190]:
y_pred = classifier.predict(X_test)

In [191]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9826657912015758


In [192]:
# Save DataFrame to csv to import to PostgreSQL and then connect back to this file (at top)
VAERS_DATA_VAX_all_df.to_csv("vaers_cleaned.csv")

In [193]:
#####

In [194]:
## Logistic Regression Model to predict death using DataFrame connected to PostgreSQL Database
y = VAERS_incidents_postgres["DIED"].copy()
X = VAERS_incidents_postgres.drop(["DIED"], axis=1).copy()
X.head()

,VAERS_ID,AGE_YRS,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,GENDER_0,GENDER_1
PRIMARY_KEY,,,,,,,,,
1,855017,55,0,1,1,1,1,1,0
2,855018,68,0,1,1,1,1,1,0
3,855019,50,0,0,0,0,0,1,0
4,855020,67,0,1,1,1,1,1,0
5,855021,73,0,0,0,0,0,1,0


In [195]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y, random_state=1, stratify=y)

In [196]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
   max_iter=200,
   random_state=1)

In [197]:
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [198]:
y_pred = classifier.predict(X_test)

In [199]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9826657912015758


In [200]:
## Logistic Regression Model to predict Hospitalization using DataFrame connected to PostgreSQL Database
y = VAERS_incidents_postgres["HOSPITAL"].copy()
X = VAERS_incidents_postgres.drop(["HOSPITAL"], axis=1).copy()
X.head()

,VAERS_ID,AGE_YRS,DIED,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,GENDER_0,GENDER_1
PRIMARY_KEY,,,,,,,,,
1,855017,55,0,1,1,1,1,1,0
2,855018,68,0,1,1,1,1,1,0
3,855019,50,0,0,0,0,0,1,0
4,855020,67,0,1,1,1,1,1,0
5,855021,73,0,0,0,0,0,1,0


In [201]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y, random_state=1, stratify=y)

In [202]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
   max_iter=200,
   random_state=1)

In [203]:
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [204]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9370978332239002


In [205]:
###########################################################################################################################

In [206]:
# Rough Neural Network
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf



In [207]:
# Split our preprocessed data into our features and target arrays
y = VAERS_incidents_postgres["DIED"].values == 1
X = VAERS_incidents_postgres.drop(["DIED"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [208]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [209]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  24
hidden_nodes_layer2 = 14

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 24)                240       
_________________________________________________________________
dense_4 (Dense)              (None, 14)                350       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 15        
Total params: 605
Trainable params: 605
Non-trainable params: 0
_________________________________________________________________


In [210]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [211]:
  # Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)  

Epoch 1/100
1428/1428 [==============================] - 2s 876us/step - loss: 551.4669 - accuracy: 0.9636
Epoch 2/100
1428/1428 [==============================] - 1s 900us/step - loss: 586.0392 - accuracy: 0.9617
Epoch 3/100
1428/1428 [==============================] - 1s 949us/step - loss: 500.0477 - accuracy: 0.9665
Epoch 4/100
1428/1428 [==============================] - 1s 886us/step - loss: 482.2612 - accuracy: 0.9647
Epoch 5/100
1428/1428 [==============================] - 1s 875us/step - loss: 430.6843 - accuracy: 0.9660
Epoch 6/100
1428/1428 [==============================] - 1s 1ms/step - loss: 394.0654 - accuracy: 0.9652: 0s - loss: 391
Epoch 7/100
1428/1428 [==============================] - 1s 948us/step - loss: 328.3452 - accuracy: 0.9678
Epoch 8/100
1428/1428 [==============================] - 1s 958us/step - loss: 321.5725 - accuracy: 0.9655
Epoch 9/100
1428/1428 [==============================] - 1s 949us/step - loss: 302.7646 - accuracy: 0.9667
Epoch 10/100
1428/1428 

1428/1428 [==============================] - 1s 877us/step - loss: 0.0865 - accuracy: 0.9829
Epoch 78/100
1428/1428 [==============================] - 1s 873us/step - loss: 0.0869 - accuracy: 0.9828
Epoch 79/100
1428/1428 [==============================] - 1s 883us/step - loss: 0.0889 - accuracy: 0.9823
Epoch 80/100
1428/1428 [==============================] - 1s 876us/step - loss: 0.0854 - accuracy: 0.9832
Epoch 81/100
1428/1428 [==============================] - 1s 899us/step - loss: 0.0899 - accuracy: 0.9821
Epoch 82/100
1428/1428 [==============================] - 1s 969us/step - loss: 0.0848 - accuracy: 0.9833
Epoch 83/100
1428/1428 [==============================] - 2s 1ms/step - loss: 0.0876 - accuracy: 0.9826
Epoch 84/100
1428/1428 [==============================] - 1s 1ms/step - loss: 0.0855 - accuracy: 0.9831
Epoch 85/100
1428/1428 [==============================] - 2s 1ms/step - loss: 0.0860 - accuracy: 0.9830
Epoch 86/100
1428/1428 [==============================] - 1s 965u

In [212]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

476/476 - 0s - loss: 0.0861 - accuracy: 0.9830
Loss: 0.08614791929721832, Accuracy: 0.9829940795898438
